## **Experimento A: Fujo de Check Out de vuelos**
### A/B Testing Convencional

#### **Escenario**
Evaluación de un experimento en el flujo de Checkout de Vuelos mediante un 'upselling' agresivo de equipaje extra antes de pagar.

__Hipótesis:__ El upselling aumentrá el Revenue, pero se teme que la fricción baje la Conversión.

_**Variante A:**_ Flujo Estándar (Control).

_**Variante B:**_ Upselling agresivo de equipaje extra antes de pagar (Tratamiento).

In [ ]:
### A - Generación de Datos Sintéticos tipo OTA (Online Travel Agency)

import pandas as pd
import numpy as np
from scipy import stats
import random


np.random.seed(42) 
n_samples = 100000

# 1. Crear asignación de variantes A y B.
# Se simnula una asignación aleatoria de usuarios a cada variante,
# enfoque estándar en las pruebas A/B para garantizar una comparación no sesgada.
variants = np.random.choice(['A', 'B'], size=n_samples, p=[0.5, 0.5])

# 2. Simular conversión (Distribución de Bernoulli)
# Grupo A: 5.1% conversión | Grupo B: 4.9% (bajó un poco por la fricción del upselling).
# Simulación con distribución de Bernoulli (1, p), 1: convirtió/compró con probabilidad p,
# 0: no convirtió/no compró con probabilidad 1-p. 
# conversions es una lista de 100000 valores de 0 y 1, donde aproximadamente el 5.1% A y el 4.9% de B tienen 1.
conv_rates = {'A': 0.051, 'B': 0.045} 
conversions = [np.random.binomial(1, conv_rates[v]) for v in variants]

# 3. Simular Revenue (Solo los que conveirtieron)
# El Revenue en travel suele ser log-normal con cola larga a la derecha (skewness positiva).
revenue = []
for i in range(n_samples):
    if conversions[i] == 0:
        rev = 0
    else:
        # AOV base $300. El grupo B gasta un poco más ($325) por el equipaje extra.
        mu = 300 if variants[i] == 'A' else 325
        rev = np.random.lognormal(mean=np.log(mu), sigma=1) # Se puede colocar un sigma más alto para más variabilidad.
    revenue.append(rev)

# 4. Crear DataFrame
df = pd.DataFrame({
    'user_id': range(n_samples), # Identificación única y trazabilidad: df[df['user_id'] == 4523].
    'variant': variants,
    'converted': conversions,
    'revenue': revenue
})

# Adicion de ruido realista
# Caso 1: error de sistema, revenue negatico por bug de logging.
df.loc[0:4, 'revenue'] = -100
# Caso 2: outliers masivos, agencias de viajes que compran paquetes corporativos.
df.loc[6:7, 'revenue'] = [50000, 75000]

print('Primeras filas del dataset original:')
print(df.head(10))
print('Muestras de filas aleatorias del dataset:')
print(df.sample(10, random_state=2588))


Primeras filas del dataset original:
   user_id variant  converted  revenue
0        0       A          0   -100.0
1        1       B          0   -100.0
2        2       B          0   -100.0
3        3       B          0   -100.0
4        4       A          0   -100.0
5        5       A          0      0.0
6        6       A          0  50000.0
7        7       B          0  75000.0
8        8       B          0      0.0
9        9       B          0      0.0
Muestras de filas aleatorias del dataset:
       user_id variant  converted     revenue
23554    23554       A          0    0.000000
72130    72130       A          0    0.000000
6430      6430       B          0    0.000000
82350    82350       A          0    0.000000
59915    59915       A          0    0.000000
12233    12233       A          0    0.000000
34587    34587       A          0    0.000000
66930    66930       A          1  107.715709
53626    53626       B          0    0.000000
8993      8993       A          

In [149]:
### B - Limpieza de Datos
# Revenue (-): imposible en una transaccion de compra, los refund se analizan y se eliminan,
# sino se verifica el log de error de transacciones para entender el origen del error.
# Outliers: se pueden eliminar o analizar por separado, dependiendo del contexto del negocio.

df_clean = df[df['revenue'] >= 0].copy() # Eliminar filas con revenue negativo.

# Detección de outliers (Método de Percentil 99 para revenue > 0)
buyers = df_clean[df_clean['converted'] == 1]
cap_value = np.percentile(buyers['revenue'], 99)

print(f"Corte de Outliers (P99): ${cap_value:.2f}")

# Opcion 1: Excluir (Trimming), agresivo, reduce muestra y puede sesgar resultados.
# df_clean = df_clean[df_clean['revenue'] <= cap_value]

# Opci[on 2: Imputar (Capping, Winsorizing)], prefereible para no perder conversiones.
# Si gasto 50000 o mas, se baja al valor del P99 (cap_value=966.31).
df_clean.loc[df_clean['revenue'] > cap_value, 'revenue'] = cap_value

Corte de Outliers (P99): $3018.02


In [ ]:
### C - Métricas
# Se analizan métricas claves como Traffic, Conversion Rate (CR), y Revenue per User (RPU).

metrics = df_clean.groupby('variant').agg({  
# agg (aggregate) aplica diferentes funciones de afgregacion a distintas columnas en una sola llamada. 
# El diccionario le indica a Pandas que calcular en cada columna:

    'user_id': 'count',            # Traffic (Denominador), cuentas columnas por varainte, numeros de usuarios por grupo.
    'converted': 'mean',           # Conversion Rate: CR = compradores/n (converted es 1 o 0, entonces el promedio es CR).
    'revenue': ['mean', 'std']     # RPU (Revenue Per User) y su varianza. RPU (mean) = ingresos totales / número de usuarios. 
                                   # std: desviación estándar, medida de dispersión, necesaria para tests de significancia (t-tests).
})

# Renombrar columnas para claridad
metrics.columns = ['traffic', 'conversion_rate', 'rpu', 'rpu_std']
metrics['TOTAL_REVENUE'] = metrics['traffic'] * metrics['rpu']

# Redondear: conversion_rate a 4 decimales, el resto a 2 decimales
metrics = metrics.round({
    'traffic': 0,
    'conversion_rate': 4,
    'rpu': 2,
    'rpu_std': 2,
    'TOTAL_REVENUE': 2
})

print("\n--- Métricas por Variante --- ")
print(metrics)


--- Métricas por Variante --- 
         traffic  conversion_rate    rpu  rpu_std  TOTAL_REVENUE
variant                                                         
A          49932           0.0510  23.44   152.83     1170571.34
B          50063           0.0465  24.01   164.08     1201974.98


##### Aclaracion de porque cambian los CR

conv_rates = {'A': 0.051, 'B': 0.049}: definen las probabilidades teóricas utilizadas para generar los datos mediante la distribución binomial np.random.binomial(1, p).

Las tasas de conversión reales observadas en la muestra simulada fluctuarán en torno a esos valores debido a la variabilidad del muestreo aleatorio. Además, se eliminaron filas con valores negativos de Revenue lo que reduce el número de muestras n.



In [ ]:
### D - Infenerencia Estadística Frecuentista (El Test Real)
# 1. Test de Conversión (Proporciones, Z-Test)

from statsmodels.stats.proportion import proportions_ztest # Este test compara dos proporciones (CR de A vs CR de B).
                                                           # para determinar si la diferencia es estadísiticament significativa o si es ruido.

successes = df_clean.groupby('variant')['converted'].sum() # Suma la columna  'converted' (0, 1) Número de conversiones por variante (éxitos).
print(f'Número de conversiones por variante:\n{successes}')

n_obs = df_clean.groupby('variant')['user_id'].count()      # Número de usuarios por variante
print(f'\nNúmero de usuarios por variante: \n{n_obs}')

stat, p_value_conv = proportions_ztest([successes['B'], successes['A']], 
                                       [n_obs['B'], n_obs['A']])

# Devuelve: stat: Z-statistic) y p_value_conv: p-value.
print(f'\nZ-statistic: {stat:.4f}, p-value: {p_value_conv:.4f}')

if p_value_conv < 0.05:
    print(">> Diferencia SIGNIFICATIVA en Conversión.")                 # Rechazar H0: la diferencia en el CR es real.
else:
    print(">> No hay diferencia significativa en Conversión (Ruido).")  # No se rechaza H0: la diferencia en el CR puede ser atribuida al ruido aleatorio.

Número de conversiones por variante:
variant
A    2546
B    2328
Name: converted, dtype: int64

Número de usuarios por variante: 
variant
A    49932
B    50063
Name: user_id, dtype: int64

Z-statistic: -3.2954, p-value: 0.0010
>> Diferencia SIGNIFICATIVA en Conversión.


El z-test calcula:

$$Z = \frac{\hat{p}_B-\hat{p}_A}{\sqrt{\hat{p}(1-\hat{p})(\frac{1}{n_b}+\frac{1}{n_A})}}$$

donde $\hat{p}$ es la proporción agrupada, como B esta primero, daria un Z negativo lo que significa que B convirtió menos que A.


In [158]:
# 2. Test de Revenue (Medias, Welch's T-Test)
# Dado que el revenue es una variable continua y puede tener varianzas diferentes entre grupos
# Welch's T-Test es una versión del t-test que no asume varianzas iguales entre los grupos, 
# lo cual es apropiado para nuestro caso con revenue, ya que la varianza de B (upselling) suele ser mayor.

rev_A = df_clean[df_clean['variant'] == 'A']['revenue']
rev_B = df_clean[df_clean['variant'] == 'B']['revenue']

t_stat, p_value_rev = stats.ttest_ind(rev_A, rev_B, equal_var=False) # Welch's T-Test, no asume que las varainza de A y B son iguales.
print(f'\nT-statistic: {t_stat:.4f}, p-value: {p_value_rev:.4f}')

if p_value_rev < 0.05:
    print(">> Diferencia SIGNIFICATIVA en Revenue.")                
else:
    print(">> No hay diferencia significativa en Revenue (Ruido).") 


T-statistic: -0.5644, p-value: 0.5725
>> No hay diferencia significativa en Revenue (Ruido).


In [167]:
# 3. Test de Mann-Whitney U (No Paramétrico)
# Dado que el revenue es una variable continua con distribución sesgada (log-normal) 
# y puede contener outliers, el test de Mann-Whitney U es una alternativa no paramétrica 
# al t-test que compara las distribuciones de dos grupos sin asumir normalidad.
# Nota: Mann-Whitney compara distribuciones (rankings), no medianas directamente.

u_stat, p_value_rev_mw = stats.mannwhitneyu(rev_A, rev_B, alternative='two-sided') # Mann-Whitney U Test, no asume normalidad ni homogeneidad de varianzas.
print(f'\nMann-Whitney U statistic: {u_stat:.2f}, p-value_mw: {p_value_rev_mw:.4f}')

if p_value_mw < 0.05:
    print(">> Diferencia SIGNIFICATIVA en Revenue.")                
else:       
    print(">> No hay diferencia significativa en Revenue (Ruido).")


Mann-Whitney U statistic: 1255319168.00, p-value_mw: 0.0014
>> Diferencia SIGNIFICATIVA en Revenue.


In [168]:
### E - Analisis de Resultados y Conclusiones

# Calculo de deltas relativos
delta_conv = (metrics.loc['B', 'conversion_rate'] - metrics.loc['A', 'conversion_rate']) / metrics.loc['A', 'conversion_rate']
delta_rpu = (metrics.loc['B', 'rpu'] - metrics.loc['A', 'rpu']) / metrics.loc['A', 'rpu']

print(f"\n--- Decisión Final ---")
print(f"Impacto en Conversión: {delta_conv:.2%}")
print(f"Impacto en Revenue/User: {delta_rpu:.2%}")

# Lógica del OEC (Overall Evaluation Criterion)
# Si se prioriza el Revenue (Profitability):
if delta_rpu > 0 and p_value_rev_mw < 0.05:   
    print("DECISIÓN: Ganadora (B). Aunque perdimos conversión, \n" 
    "el lift en ticket promedio compensa y genera más revenue.")
    
# Si se prioriza el Market Share (Conversión):
elif delta_conv < 0 and p_value_conv < 0.05:
    print("DECISIÓN: LOSER (B). Estamos perdiendo usuarios. A largo plazo esto daña la retención.")


--- Decisión Final ---
Impacto en Conversión: -8.82%
Impacto en Revenue/User: 2.43%
DECISIÓN: Ganadora (B). Aunque perdimos conversión, 
el lift en ticket promedio compensa y genera más revenue.
